In [347]:
# Import libraries
import pandas as pd
import mlflow
import sqlite3 as sql
from datetime import timedelta, datetime
import os
import plotly.express as px

In [348]:
sql.sqlite_version

'3.39.3'

# Connection data 

In [349]:
conn = sql.connect('Application_prod.db')

data = pd.read_sql("""SELECT * 
                      FROM Raw_data""", conn)
data

,index,fullVisitorId,bounces,time_on_site,pageviews,medium,channelGrouping,deviceCategory,country,date
0,0,1841398939845413817,0,11142,330.0,(none),Direct,desktop,United States,2022-10-10 00:00:00
1,1,1841398939845413817,0,11142,59.0,(none),Direct,desktop,United States,2022-10-10 00:00:00
2,2,1841398939845413817,0,11142,55.0,(none),Direct,desktop,United States,2022-10-10 00:00:00
3,3,1841398939845413817,0,11142,144.0,(none),Direct,desktop,United States,2022-10-10 00:00:00
4,4,1841398939845413817,0,11142,67.0,(none),Direct,desktop,United States,2022-10-10 00:00:00
...,...,...,...,...,...,...,...,...,...,...
87671,48049,4362490200806308515,1,0,1.0,(none),Direct,desktop,(not set),2022-09-24 00:00:00
87672,48050,6737381346243471980,1,0,1.0,(none),Direct,desktop,(not set),2022-09-24 00:00:00
87673,48051,3763043934585916260,1,0,1.0,(none),Direct,desktop,(not set),2022-09-24 00:00:00
87674,48052,1512694728925216829,1,0,1.0,(none),Direct,desktop,United States,2022-09-26 00:00:00


In [350]:
conn = sql.connect('Application_prod.db')

predict_data = pd.read_sql("""SELECT * 
                      FROM Result_data""", conn)
predict_data

,index,fullVisitorId,date,Predict
0,6,222234563178632303,2022-09-03 00:00:00,0
1,7,222234563178632303,2022-09-03 00:00:00,1
2,8,222234563178632303,2022-09-03 00:00:00,1
3,9,222234563178632303,2022-09-03 00:00:00,0
4,10,222234563178632303,2022-09-03 00:00:00,0
...,...,...,...,...
49888,48049,4362490200806308515,2022-09-24 00:00:00,0
49889,48050,6737381346243471980,2022-09-24 00:00:00,0
49890,48051,3763043934585916260,2022-09-24 00:00:00,0
49891,48052,1512694728925216829,2022-09-26 00:00:00,0


In [351]:
predict_data.index.value_counts().sort_values(ascending=False)

0        1
21       1
24       1
23       1
11       1
        ..
49886    1
49887    1
49888    1
49889    1
49892    1
Length: 49893, dtype: int64

In [352]:
data.index.value_counts().sort_values(ascending=False)

0        1
12       1
4        1
5        1
6        1
        ..
87674    1
87657    1
87656    1
87655    1
87675    1
Length: 87676, dtype: int64

In [353]:
data["id"]= data['index']
predict_data['id'] = predict_data["index"]

In [354]:
predict_data[["id","Predict"]]

,id,Predict
0,6,0
1,7,1
2,8,1
3,9,0
4,10,0
...,...,...
49888,48049,0
49889,48050,0
49890,48051,0
49891,48052,0


In [355]:
d = data.set_index("index")
p = predict_data.set_index("index")
data_new = d.merge(p)

In [356]:
data_new

,fullVisitorId,bounces,time_on_site,pageviews,medium,channelGrouping,deviceCategory,country,date,id,Predict
0,951958281389593717,0,5192,490.0,(none),Direct,desktop,France,2022-10-01 00:00:00,75,0
1,6479505754536046846,0,2005,2.0,referral,Referral,mobile,United States,2022-10-01 00:00:00,1257,0
2,7977975644602008880,0,1834,26.0,organic,Organic Search,desktop,United States,2022-10-01 00:00:00,1598,1
3,7977975644602008880,0,1834,4.0,organic,Organic Search,desktop,United States,2022-10-01 00:00:00,1599,0
4,2587418382449492614,0,1357,10.0,organic,Organic Search,desktop,United States,2022-10-01 00:00:00,2680,1
...,...,...,...,...,...,...,...,...,...,...,...
49888,4362490200806308515,1,0,1.0,(none),Direct,desktop,(not set),2022-09-24 00:00:00,48049,0
49889,6737381346243471980,1,0,1.0,(none),Direct,desktop,(not set),2022-09-24 00:00:00,48050,0
49890,3763043934585916260,1,0,1.0,(none),Direct,desktop,(not set),2022-09-24 00:00:00,48051,0
49891,1512694728925216829,1,0,1.0,(none),Direct,desktop,United States,2022-09-26 00:00:00,48052,0


In [359]:
import streamlit as st

In [ ]:
st.

In [357]:
#data_new.to_csv("data_app.csv", index=False)

# Top 10 Nombre pageviews par pays

In [367]:
df_pie_us_vs_all = data_new.copy()[data_new['country']!="(not set)"]
df_pie_us_vs_all['country'] = df_pie_us_vs_all["country"].apply(lambda x: "United States" if x == "United States" else "other")

In [369]:
df_pie_undifined_country = data_new.copy()
df_pie_undifined_country['country'] = df_pie_undifined_country["country"].apply(lambda x: "Non-définie" if x == "(not set)" else "Définie")

In [376]:
list_of_country = [[37.09024,-95.712891,"United States"],
[20.593684,78.96288,"India"],
[46.227638,2.213749,"France"],
[35.86166,104.195397,"China"],
[61.52401,105.318756,"Russia"],
[51.165691,10.451526,"Germany"],
[61.92411,25.748151,"Finland"],
[55.378051,3.435973,"United Kingdom"],
[52.132633,5.291266,"Netherlands"],
[56.130366,106.346771,"Canada"]]
df3 = pd.DataFrame(list_of_country, columns = ["Latitude", 'Longitude', 'country'])


In [379]:
data_with_coordonate = data_new.copy().merge(df3,on ="country",how = "left")

In [380]:
data_for_map = data_with_coordonate[data_with_coordonate['country'] != "(not set)"]

In [386]:
data_for_map_grouped = data_for_map.groupby("country").agg({"fullVisitorId": ["count","nunique"],
                                    "time_on_site": ["mean","sum","max","min"],
                                    "pageviews": ['mean','sum',"max","min"],
                                    "Latitude": ['first'],
                                    "Longitude": ['first']})

In [426]:
fig = px.scatter_mapbox(lat = data_for_map_grouped["Latitude"]['first'],lon = data_for_map_grouped["Longitude"]['first'],size=data_for_map_grouped["time_on_site"]['mean'],color=data_for_map_grouped["pageviews"]['mean'],color_continuous_scale=px.colors.sequential.Viridis,mapbox_style ='open-street-map',size_max=50,zoom=1)

fig.show()

In [ ]:
mapbox_style: str (default `'basic'`, needs Mapbox API token)
    Identifier of base map style, some of which require a Mapbox API token
    to be set using `plotly.express.set_mapbox_access_token()`. Allowed
    values which do not require a Mapbox API token are `'open-street-map'`,
    `'white-bg'`, `'carto-positron'`, `'carto-darkmatter'`, `'stamen-
    terrain'`, `'stamen-toner'`, `'stamen-watercolor'`. Allowed values
    which do require a Mapbox API token are `'basic'`, `'streets'`,
    `'outdoors'`, `'light'`, `'dark'`, `'satellite'`, `'satellite-
    streets'`.

In [ ]:
data_for_map.groupby("country").

In [149]:
df_pie_country = data_new.copy()[data_new["country"].isin(["United States", "France", "Russia",
                                                       "India", "China", "Germany", "Finland",
                                                       "Canada", "Netherlands", "United Kingdom"])]
df_pie_country['country'].value_counts(ascending=False)

United States    39555
India              239
Canada             142
Germany            117
France             102
China               63
Name: country, dtype: int64

In [150]:
df_pie_country['country']

0               France
1        United States
2        United States
3        United States
4        United States
             ...      
49879    United States
49880    United States
49882    United States
49886    United States
49891    United States
Name: country, Length: 40218, dtype: object

In [151]:
df_pie_country.groupby("country").sum()

,bounces,time_on_site,pageviews,id,Predict
country,,,,,
Canada,72,19257,409.0,4280694,10
China,59,2028,83.0,2422960,0
France,60,144635,10770.0,2626739,3
Germany,101,4344,153.0,4238325,1
India,107,47729,820.0,6753369,19
United States,7182,18102448,272934.0,792661962,13410


In [152]:
fig = px.pie(df_pie_country, values='pageviews', names='country', title='Top 10 Nombre pageviews par pays')
fig.show()

# Top 10 d'utilisateurs uniques par pays

In [153]:
df_unique_user_country = data_new.copy().groupby(['country', 'fullVisitorId']).count().reset_index().groupby("country").count().reset_index()[["country","fullVisitorId"]]

In [154]:
df_unique_user_country = df_unique_user_country.sort_values('fullVisitorId', ascending=False).head(10)

In [155]:
df_unique_user_country.groupby(['country']).count().reset_index()

,country,fullVisitorId
0,(not set),1
1,Canada,1
2,China,1
3,France,1
4,Germany,1
5,India,1
6,United States,1


In [156]:
figure = px.pie(df_unique_user_country, values='fullVisitorId', names='country', title='Top 10 des visiteurs uniques par pays')
figure.show()

# Top des Visiteurs

In [157]:
df_user = data_new.copy().groupby(['fullVisitorId']).count()

In [158]:
df_top_user = df_user.sort_values('will_buy_on_return_visit', ascending=False).head(10)

KeyError: 'will_buy_on_return_visit'

In [ ]:
df_top_user

In [84]:
df_top_user["count"] = df_top_user['bounces']
df_top_user = df_top_user["count"].reset_index()

In [97]:
df_top_user["fullVisitorId"] = df_top_user['fullVisitorId'].astype(str)

In [98]:
fig = px.bar(df_top_user.sort_values("count"), x= 'count', y= 'fullVisitorId')
fig.show()

# Nombre de prédiction de la semaine

In [310]:
top_predict_by_week = data_new.copy()
top_predict_by_week["date"] = pd.to_datetime(top_predict_by_week["date"])

In [311]:
top_predict_by_week = top_predict_by_week[['date', 'Predict']]

In [313]:
date = "20220917"

today_date = datetime.strptime(date,"%Y%m%d")
last_date = today_date - timedelta(days=7)

top_predict_this_week = top_predict_by_week.copy()[top_predict_by_week['date'].between(last_date,today_date)]

In [317]:
date = "20220909"

today_date = datetime.strptime(date,"%Y%m%d")
last_date = today_date - timedelta(days=7)


top_predict_last_week = top_predict_by_week.copy()[top_predict_by_week['date'].between(last_date,today_date)]

In [320]:
count_predict_this_week = top_predict_this_week.count()[0]
count_predict_last_week = top_predict_last_week.count()[0]

In [333]:
count_predict_last_week

14901

In [338]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = count_predict_this_week,
    domain = {'x': [0, 0], 'y': [0, 0]},
    delta = {'reference': count_predict_last_week, 'relative': True, 'position' : "bottom",'valueformat':'.2%'}))

# fig.add_trace(go.Indicator(
#     mode = "number+delta",
#     value = 350,
#     delta = {'reference': 400, 'relative': True},
#     domain = {'x': [0, 0.5], 'y': [0.5, 1]}))

# fig.add_trace(go.Indicator(
#     mode = "number+delta",
#     value = 450,
#     title = {"text": "Accounts<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>"},
#     delta = {'reference': 400, 'relative': True},
#     domain = {'x': [0.6, 1], 'y': [0, 1]}))

fig.show()